In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("tomthescientist/netflix-facebook-posts-as-sentences-for-llm-input")

# print("Path to dataset files:", path)

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # '2' to suppress INFO and WARNING logs, '3' to suppress ERROR as well
import tensorflow as tf

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Check if GPU is being used
devices = tf.config.list_physical_devices('GPU')
if len(devices) > 0:
    print(f"GPU is being used: {devices}")
else:
    print("GPU is not being used.")

physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    try:
        # Set memory growth to avoid TensorFlow allocating all GPU memory
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("Memory growth enabled for GPU.")
    except RuntimeError as e:
        print(e)

# tf.debugging.set_log_device_placement(True)

Num GPUs Available:  0
GPU is not being used.


In [27]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input, Dropout
import os

In [5]:
os.chdir('/Users/ankeshkumar/.cache/kagglehub/datasets/tomthescientist/netflix-facebook-posts-as-sentences-for-llm-input/versions/3')

In [6]:
os.listdir()

['netflix_fb_sentences.csv', 'netflix_fb_comments.csv']

In [7]:
import pandas as pd
df = pd.read_csv('netflix_fb_sentences.csv')
df = df[:50000]
df['Sentence'].head()

0    Raymond Stewart commented "Why isn't everyone ...
1    Nichole Bradley commented "The CGI in Tiny Pre...
2    Jamie Shields commented "The characters in Gen...
3    Summer Mahoney commented "I wish the supportin...
4    Barbara Webb commented "Netflix really knows h...
Name: Sentence, dtype: object

In [8]:
df.describe()

,Sentence
count,50000
unique,50000
top,"Raymond Stewart commented ""Why isn't everyone ..."
freq,1


In [9]:
df['Sentence'].isnull().sum()

0

In [10]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

# Ensure stopwords are downloaded
nltk.download('stopwords')

# Initialize stemmer and corpus
ps = PorterStemmer()
corpus = []

# Ensure the DataFrame index is continuous
if not df.index.is_monotonic_increasing or not df.index.is_integer():
    df.reset_index(drop=True, inplace=True)

# Filter missing content
df = df[df['Sentence'].notna()]

# Iterate over the content
for i in range(len(df)):
    try:
        review = re.sub('[^a-zA-Z]', ' ', str(df['Sentence'].iloc[i]))
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
        corpus.append(' '.join(review))
    except Exception as e:
        print(f"Error processing row {i}: {e}")
 
print("Corpus creation completed.")

[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
/var/folders/mv/3g_5nl353bl_10h4z7l_h8f80000gn/T/ipykernel_11863/428114608.py:14: FutureWarning: RangeIndex.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  if not df.index.is_monotonic_increasing or not df.index.is_integer():


Corpus creation completed.


In [11]:
corpus[:10]

['raymond stewart comment everyon talk daybreak break new ground facebook',
 'nichol bradley comment cgi tini pretti thing groundbreak actor outshin other facebook',
 'jami shield comment charact gentefi wonder develop actor outshin other facebook',
 'summer mahoney comment wish support charact ragnarok screen time potenti facebook',
 'barbara webb comment netflix realli know tell stori last kingdom beauti craft facebook',
 'thoma rey comment believ peopl watch sens need recognit facebook',
 'evelyn santo comment bridgerton hand best netflix seri plot twist keep edg seat facebook',
 'mark rio comment wait talk luke cage everyon share experi facebook',
 'stephen simmon comment charact arc lockwood co beauti develop everi charact feel real facebook',
 'crystal munoz comment anyon els feel like gentefi end soon controversi facebook']

In [12]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)  # Fit the tokenizer on the entire corpus
total_words = len(tokenizer.word_index) + 1  # Total unique words + 1 for padding

In [13]:
# tokenizer.word_index

In [14]:
from collections import Counter
word_counts = Counter(" ".join(corpus).split())
print(word_counts.most_common(10))  # Check the most frequent words

import random

balanced_corpus = []
for sentence in corpus:
    words = sentence.split()
    if words.count("comment") > 0:
        words = [word for word in words if word != "comment"] + random.sample(["comment"] * 10, min(10, words.count("comment")))
    if words.count("facebook") > 0:
        words = [word for word in words if word != "facebook"] + random.sample(["facebook"] * 10, min(10, words.count("facebook")))
    balanced_corpus.append(" ".join(words))

[('comment', 50000), ('facebook', 50000), ('netflix', 8588), ('watch', 6670), ('charact', 5784), ('everi', 4603), ('like', 4479), ('see', 4229), ('feel', 4151), ('need', 3329)]


In [15]:
# tokenizer.word_index

In [16]:
# Create input-output pairs (n-grams)
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]  # Convert words to indices
    for i in range(1, len(token_list)):  # Create n-gram sequences
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

In [17]:
max_sequence_len = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
input_sequences = np.array(input_sequences)

X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = np.eye(total_words)[y]  # One-hot encode the labels

In [18]:
X = X.astype(np.float32)
y = y.astype(np.float32)

In [19]:
X.shape

(524000, 20)

In [20]:
y.shape

(524000, 2137)

In [21]:
# from tensorflow.keras.utils import to_categorical

# # Assuming `y` contains integer class indices
# y = to_categorical(y, num_classes=2137)

# batch_size = 64  # Batch size for training
# train_dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size).shuffle(buffer_size=10000)

# for x_batch, y_batch in train_dataset.take(1):
#     print("Input Batch Shape:", x_batch.shape)  # Should be (64, 20)
#     print("Label Batch Shape:", y_batch.shape)  # Should be (64, 2137)

In [22]:
input_sequences[:, :-1]

array([[  0,   0,   0, ...,   0,   0, 729],
       [  0,   0,   0, ...,   0, 729, 555],
       [  0,   0,   0, ..., 729, 555,   1],
       ...,
       [  0,   0,   0, ..., 666, 102, 313],
       [  0,   0,   0, ..., 102, 313,  21],
       [  0,   0,   0, ..., 313,  21,  22]], dtype=int32)

In [23]:
input_sequences[:, -1]

array([555,   1,  37, ...,  21,  22,   2], dtype=int32)

In [24]:
input_sequences

array([[  0,   0,   0, ...,   0, 729, 555],
       [  0,   0,   0, ..., 729, 555,   1],
       [  0,   0,   0, ..., 555,   1,  37],
       ...,
       [  0,   0,   0, ..., 102, 313,  21],
       [  0,   0,   0, ..., 313,  21,  22],
       [  0,   0,   0, ...,  21,  22,   2]], dtype=int32)

In [25]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

## Build the model

In [28]:
# Define the model with Dropout layers
model = Sequential([
    Embedding(input_dim=total_words, output_dim=50, input_length=20),  # Match input sequence length (20)
    Bidirectional(LSTM(100, return_sequences=False)),
    Dropout(0.2),  # Apply dropout with 20% rate
    Dense(total_words, activation='softmax')  # Match total words (2137)
])

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              metrics=['accuracy'])

# Example input for model building (e.g., batch size 64, sequence length 20)
example_input = np.zeros((64, 20))  # Batch size 64, sequence length 20
model.build(example_input.shape)

# Print model summary
print(model.summary())

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (64, 20, 50)           │       106,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (64, 200)              │       120,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (64, 200)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 2137)             │       429,537 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 657,187 (2.51 MB)

 Trainable params: 657,187 (2.51 MB)

 Non-trainable params: 0 (0.00 B)

None


In [29]:
early_stopper = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0,
)

In [ ]:
history = model.fit(X, y, validation_split=0.33, epochs=10, batch_size=64, callbacks=[early_stopper], verbose=1)

Epoch 1/10
5486/5486 ━━━━━━━━━━━━━━━━━━━━ 206s 37ms/step - accuracy: 0.2869 - loss: 4.2790 - val_accuracy: 0.5528 - val_loss: 2.2014
Epoch 2/10
5486/5486 ━━━━━━━━━━━━━━━━━━━━ 211s 38ms/step - accuracy: 0.5609 - loss: 2.1192 - val_accuracy: 0.5927 - val_loss: 1.8401
Epoch 3/10
5486/5486 ━━━━━━━━━━━━━━━━━━━━ 240s 44ms/step - accuracy: 0.5894 - loss: 1.8470 - val_accuracy: 0.5961 - val_loss: 1.7585
Epoch 4/10
5486/5486 ━━━━━━━━━━━━━━━━━━━━ 225s 41ms/step - accuracy: 0.5974 - loss: 1.7517 - val_accuracy: 0.5997 - val_loss: 1.7108
Epoch 5/10
2029/5486 ━━━━━━━━━━━━━━━━━━━━ 1:50 32ms/step - accuracy: 0.6017 - loss: 1.6899

## Text generation function

In [ ]:
seed_text = input()
next_words = 4

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = tf.keras.utils.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    
    # Debugging: Check the predicted probabilities and the word
    print("Predicted Probabilities:", predicted[0])
    predicted_index = predicted.argmax()
    print("Predicted Index:", predicted_index)
    print(predicted[0])
    
    predicted_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            predicted_word = word
            break
    
    if not predicted_word:
        print("Error: Predicted word not found!")
        break
    
    seed_text += " " + predicted_word

print("Generated Text:", seed_text)

In [ ]:
import matplotlib.pyplot as plt

# Plot training accuracy
plt.plot(history.history['accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [ ]:
# Plot training loss
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()